In [2]:
# All Libraries imported

import pandas as pd
import urllib.request
import requests
from xml.etree import ElementTree as ET
import datetime
format = "%d/%m/%Y %H:%M"
import seaborn as sns
import matplotlib.pyplot as plt
import urllib.request
import json
import numpy as np
import urllib
from darksky import forecast
import datetime
import forecastio
import getpass
import matplotlib.pyplot as plt


In [2]:
# Historical Bike Journey Data

## Read list of names of all files from a separate CSV
with open('tfl_bike_trips.csv', 'r') as f:
    csv_list = f.read().splitlines()

## Downloading all of the bike journey CSV files for 2019 and appending to one dataset
website = 'http://cycling.data.tfl.gov.uk/usage-stats/'

url_list = [website + urllib.parse.quote(x) for x in csv_list]
dfs = (pd.read_csv(url) for url in url_list)
bike_journey_data = pd.concat(dfs, ignore_index=True)

bike_journey_data['Start Date']= bike_journey_data['Start Date']
bike_journey_data['End Date']=bike_journey_data['End Date']
bike_journey_data['Start Date Converted']= pd.to_datetime(bike_journey_data['Start Date'], format=format).dt.date
bike_journey_data['End Date Converted']= pd.to_datetime(bike_journey_data['End Date'], format=format).dt.date
bike_journey_data['Hours']= pd.to_datetime(bike_journey_data['Start Date'], format=format).dt.hour
bike_journey_data['Week Day']= pd.to_datetime(bike_journey_data['Start Date'], format=format).dt.weekday
bike_journey_data['Day']= pd.to_datetime(bike_journey_data['Start Date'], format=format).dt.day
bike_journey_data['Month']= pd.to_datetime(bike_journey_data['Start Date'], format=format).dt.month
bike_journey_data['Year']= pd.to_datetime(bike_journey_data['Start Date'], format=format).dt.year
bike_journey_data['Duration in minutes']=bike_journey_data['Duration']/60
bike_journey_data['id'] = bike_journey_data['Year'].map(str) + '-' + bike_journey_data['Month'].map(str) + '-' + bike_journey_data['Day'].map(str)
bike_journey_data["id"] = bike_journey_data["id"].astype(str)
bike_journey_data['id_hours'] = bike_journey_data['Start Date Converted'].map(str)+ '-' + bike_journey_data['Hours'].map(str)
bike_journey_data["id_hours"] = bike_journey_data["id_hours"].astype(str)

bike_journey_data.to_csv('bike_journey_data_27_mar_2020.csv', header=True, index=True)

NameError: name 'df' is not defined

In [3]:
# Real time Bike location data

locu_api='0562cc774cc1e0842794be372d6fd545'

url='https://api.tfl.gov.uk/BikePoint?app_id=bd90e5e6&app_key=0562cc774cc1e0842794be372d6fd545'
json_obj=urllib.request.urlopen(url)

data=json.load(json_obj)

value1 =[]
for item1 in range(len(data)):
    for item2 in range(len(data[item1]['additionalProperties'])):
        if (data[item1]['additionalProperties'][item2]['key'])=="NbDocks":
            value1.append(data[item1]['additionalProperties'][item2]['value'])

value2 =[]
for item1 in range(len(data)):
    for item2 in range(len(data[item1]['additionalProperties'])):
        if (data[item1]['additionalProperties'][item2]['key'])=="NbEmptyDocks":
            value2.append(data[item1]['additionalProperties'][item2]['value'])
            
value3 =[]
for item1 in range(len(data)):
     value3.append(data[item1]['commonName'])
        
value4 =[]
for item1 in range(len(data)):
     value4.append(data[item1]['lat'])
        
value5 =[]
for item1 in range(len(data)):
     value5.append(data[item1]['lon'])
        

value6 =[]
for item1 in range(len(data)):
    for item2 in range(len(data[item1]['additionalProperties'])):
        if (data[item1]['additionalProperties'][item2]['key'])=="NbBikes":
            value6.append(data[item1]['additionalProperties'][item2]['value']) 
            
value7 =[]
for item1 in range(len(data)):
     value7.append(data[item1]['id'])

ID=[]
for value in value7:
    ID.append(value[11:])
    
         
#d = {'Bike point': value3, 'No of avail Bikes': value4, 'No empty docks' : value2, 'Total docks': value1}

bike_location_data= pd.DataFrame(list(zip(value1, value2, value3, 
                                value4, value5, value6, ID)), 
                columns = ["Capacity","Empty docks","Bike station","Lat", "Lon", "Avail bikes", "id"])

bike_location_data["id"] = bike_location_data["id"].astype(int)

bike_location_data.to_csv('bike_location_data_27_March_2020.csv', header=True, index=True)

In [2]:
# Getting the Weather Data from DarkSky

api_key = ('4bd2f4701fdba6c093dc1857f669bd23')
lat = 51.51
lng = -0.13
date = datetime.datetime(2019,1,1)
forecast = forecastio.load_forecast(api_key, lat, lng, time=date)
attributes = ["temperature", "precipIntensity", "humidity", "windSpeed", "visibility"]

times = []
data = {}
for attr in attributes:
    data[attr] = []

start = datetime.datetime(2019, 1, 1)
for offset in range(1, 365):
    forecast = forecastio.load_forecast('4bd2f4701fdba6c093dc1857f669bd23', '51.51', '-0.13', time=start+datetime.timedelta(offset), units="uk")
    h = forecast.hourly()
    d = h.data
    for p in d:
        times.append(p.time)
        try:
            for i in attributes:
                data[i].append(p.d[i])
        except:
            print(KeyError)

weather_data = pd.DataFrame(data, index=times)

weather_data.index.names = ['Date']

weather_data['Date']=weather_data.index
weather_data['Date Converted']= pd.to_datetime(weather_data['Date'], format=format).dt.date
weather_data['Hours']= pd.to_datetime(weather_data['Date'], format=format).dt.hour
weather_data["Hours"].replace(0, 24)
weather_data['Day']= pd.to_datetime(weather_data['Date'], format=format).dt.day
weather_data['Month']= pd.to_datetime(weather_data['Date'], format=format).dt.month
weather_data['Year']= pd.to_datetime(weather_data['Date'], format=format).dt.year
weather_data['id'] = weather_data['Year'].map(str) + '-' + weather_data['Month'].map(str) + '-' + weather_data['Day'].map(str).map(str)
weather_data["id"] = weather_data["id"].astype(str)
weather_data["Hours"] = weather_data["Hours"].astype(str)
weather_data=weather_data.drop(columns=['Day', 'Year', 'Month', 'Date'])
weather_data['id_Hours'] = weather_data['Date Converted'].map(str) + '-' + weather_data['Hours'].map(str)
weather_data["id_Hours"] = weather_data["id_Hours"].astype(str)

#%matplotlib inline
#plt.style.use('ggplot')
#weather_data.plot(subplots=True);

weather_data.to_csv('weather_data_27_mar_2020.csv', header=True, index=True)

In [ ]:
 ## Combine journey data with location data
    
'''bike_data= bike_journey_data.merge(right =bike_location_data,
                             how = 'inner',
                             left_on = 'StartStation Id',
                             right_on = 'id')

pd.set_option('display.max_columns', 999)
bike_data=bike_data.drop(columns=['id_y','Year'])
bike_data.sort_values(by=['id_x'])'''